In [ ]:
# Import necessary libraries
from google.colab import drive
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import numpy as np
import glob
from sklearn.model_selection import train_test_split
from PIL import Image

drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Poisson_Study/data")

In [ ]:
import numpy as np
import cv2
import glob
import pandas as pd
from skimage.transform import radon, iradon, rescale
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from skimage.io import imread
import matplotlib.pyplot as plt
from os.path import basename

In [ ]:
import numpy as np
import cv2
from skimage.io import imread
from skimage.transform import radon, iradon
from skimage.filters import gaussian
import glob
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Poisson_Study/labels.csv")

def prepare_images(file_paths, df, image_size=(256, 256), theta=np.linspace(0., 180., 180, endpoint=False), apply_sinogram=True):
    img_array = []
    labels = []
    for file_path in file_paths:
        img = imread(file_path, as_gray=True)
        if img is None:
            print("Failed to load image:", file_path)
            continue

        img = cv2.resize(img, image_size)

        if apply_sinogram:
            sinogram = radon(img, theta=theta)
            scale_factor = 0.5
            noisy_sinogram = np.random.poisson(sinogram * scale_factor)
            reconstructed_image = iradon(noisy_sinogram, theta=theta, filter_name='hamming')
            img = reconstructed_image

        img = np.expand_dims(img, axis=-1) / 255.0

        img_array.append(img)

        base_name = os.path.basename(file_path)
        label = df[df['filename'] == base_name]['hemorrhage'].values
        if len(label) > 0:
            labels.append(label[0])
        else:
            print(f"No label found for {base_name}; skipping image.")

    return np.array(img_array), np.array(labels)

file_paths = glob.glob("/content/drive/My Drive/Poisson_Study/data/*.png")

X_train, X_test = train_test_split(file_paths, test_size=0.2, random_state=8)
X_test, X_validation = train_test_split(X_test, test_size=0.5, random_state=8)
X_train_processed, y_train = prepare_images(X_train, df, apply_sinogram=True)
X_test_processed, y_test = prepare_images(X_test, df, apply_sinogram=False)
X_validation_processed, y_validation = prepare_images(X_validation, df, apply_sinogram=False)
X_train_unprocessed, y_train_unprocessed = prepare_images(X_train, df, apply_sinogram=False)
X_train_combined = np.concatenate([X_train_processed, X_train_unprocessed])
y_train_combined = np.concatenate([y_train, y_train_unprocessed])


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os
from skimage.io import imread
from skimage.transform import radon, iradon
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

class Block(tf.keras.Model):
    def __init__(self, filters, kernel_size, repetitions, pool_size=2, strides=2):
        super(Block, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.repetitions = repetitions
        self.conv_layers = [
            tf.keras.layers.Conv2D(filters, kernel_size, activation='relu', padding="same")
            for _ in range(repetitions)
        ]
        self.max_pool = tf.keras.layers.MaxPooling2D(pool_size, strides=strides)

    def call(self, inputs):
        x = inputs
        for conv in self.conv_layers:
            x = conv(x)
        return self.max_pool(x)

class MyVGG(tf.keras.Model):
    def __init__(self, num_classes):
        super(MyVGG, self).__init__()
        self.block_a = Block(filters=64, kernel_size=3, repetitions=2)
        self.block_b = Block(filters=128, kernel_size=3, repetitions=2)
        self.block_c = Block(filters=256, kernel_size=3, repetitions=3)
        self.block_d = Block(filters=512, kernel_size=3, repetitions=3)
        self.block_e = Block(filters=512, kernel_size=3, repetitions=3)
        self.flatten = tf.keras.layers.Flatten()
        self.fc = tf.keras.layers.Dense(256, activation='relu')
        self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs, return_last_conv=False):
        x = self.block_a(inputs)
        x = self.block_b(x)
        x = self.block_c(x)
        x = self.block_d(x)
        x = self.block_e(x)
        if return_last_conv:
            return x
        x = self.flatten(x)
        x = self.fc(x)
        return self.classifier(x)

def prepare_train_images(file_paths, image_size=(256, 256), theta=np.linspace(0., 180., 180, endpoint=False)):
    img_array = []
    labels = []
    for file_path in file_paths:
        base_name = os.path.basename(file_path)

        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, image_size)
            sinogram = radon(img, theta=theta)

            scale_factor = 2
            noisy_sinogram = np.random.poisson(sinogram * scale_factor)

            reconstructed_image = iradon(noisy_sinogram, theta=theta, filter_name='hamming')

            reconstructed_image = np.expand_dims(reconstructed_image, axis=-1) / 255.0
            img_array.append(reconstructed_image)

            label_series = df[df['filename'] == base_name]['hemorrhage']
            if not label_series.empty:
                labels.append(label_series.iloc[0])
            else:
                print(f"No label found for {base_name}; skipping image.")
                continue
        else:
            print("Failed to load image:", file_path)

    return np.array(img_array), np.array(labels)

def prepare_test_images(file_paths, image_size=(256, 256)):
    img_array = []
    labels = []
    for file_path in file_paths:
        base_name = os.path.basename(file_path)

        img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, image_size)
            img = np.expand_dims(img, axis=-1)
            img_array.append(img)

            label_series = df[df['filename'] == base_name]['hemorrhage']
            if not label_series.empty:
                labels.append(label_series.iloc[0])
            else:
                print(f"No label found for {base_name}; skipping image.")
                continue
        else:
            print("Failed to load image:", file_path)

    return np.array(img_array) / 255.0, np.array(labels)


df = pd.read_csv("/content/drive/MyDrive/Poisson_Study/labels.csv")
file_paths = glob.glob("/content/drive/My Drive/Poisson_Study/data/*.png")
X_train_files, X_temp = train_test_split(file_paths, test_size=0.2, random_state=8)
X_validation_files, X_test_files = train_test_split(X_temp, test_size=0.5, random_state=8)
X_train_processed, y_train = prepare_train_images(X_train_files)

X_test, y_test = prepare_test_images(X_test_files)
X_validation, y_validation = prepare_test_images(X_validation_files)

X_train_combined, y_train_combined = prepare_train_images(X_train_files)
learning_rate = 0.001
beta_1 = 0.9
beta_2 = 0.999
epsilon = 1e-07

optimizer = Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, epsilon=epsilon)
model = MyVGG(num_classes=2)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_combined, y_train_combined, epochs=20, validation_data=(X_validation, y_validation))
